In [34]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import colorlover as cl

# 解决plotly离线模式下不出图

In [35]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [36]:
fname = 'BlackFriday.csv'
data = pd.read_csv(fname)
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


#  查看表中关键字Occupation列中不重复的项，即不同职业的个数，返回一个列表

In [37]:
data.Occupation.unique()

array([10, 16, 15,  7, 20,  9,  1, 12, 17,  0,  3,  4, 11,  8, 19,  2, 18,
        5, 14, 13,  6], dtype=int64)

# 有多少种职业和多少人参与了本次活动

In [38]:
len(data.Occupation.unique())

21

In [39]:
len(data.User_ID.unique())

5891

# 查看表的统计信息

In [40]:
data.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.375770e+05,537577.00000,537577.000000,537577.000000,370591.000000,164278.000000,537577.000000
mean,1.002992e+06,8.08271,0.408797,5.295546,9.842144,12.669840,9333.859853
std,1.714393e+03,6.52412,0.491612,3.750701,5.087259,4.124341,4981.022133
min,1.000001e+06,0.00000,0.000000,1.000000,2.000000,3.000000,185.000000
25%,1.001495e+06,2.00000,0.000000,1.000000,5.000000,9.000000,5866.000000
50%,1.003031e+06,7.00000,0.000000,5.000000,9.000000,14.000000,8062.000000
75%,1.004417e+06,14.00000,1.000000,8.000000,15.000000,16.000000,12073.000000
max,1.006040e+06,20.00000,1.000000,18.000000,18.000000,18.000000,23961.000000


# 查看表中各个字段有多少缺失值

In [41]:
data.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            166986
Product_Category_3            373299
Purchase                           0
dtype: int64

# 统计每个消费者的性别及本次活动的花费

In [42]:
# pivot_table: 数据透视表
gender_purchase = data.pivot_table(values="Purchase", index=["User_ID", "Gender"], aggfunc="sum")
gender_purchase = gender_purchase.reset_index()
print(gender_purchase.head(10))

   User_ID Gender  Purchase
0  1000001      F    333481
1  1000002      M    810353
2  1000003      M    341635
3  1000004      M    205987
4  1000005      M    821001
5  1000006      F    379450
6  1000007      M    234427
7  1000008      M    796545
8  1000009      M    593960
9  1000010      F   2169486


# 统计有多少个消费者

In [43]:
gender_purchase.count()

User_ID     5891
Gender      5891
Purchase    5891
dtype: int64

# 统计男性和女性消费者在本次活动的占比情况

In [44]:
gender_count = gender_purchase.groupby(by="Gender").size().reset_index(name="人数")
gender_count['占比'] = gender_count['人数']/gender_count['人数'].sum()
gender_count

,Gender,人数,占比
0,F,1666,0.282804
1,M,4225,0.717196


# 用饼图可视化男人和女人消费的对比情况

In [45]:
trace = go.Pie(labels = gender_purchase.Gender.to_list(),
              values = gender_purchase.Purchase.to_list(),
              hole=0.6)
layout = go.Layout(title="the comparison of consumption between men and women during BlackFriday")
fig = go.Figure(data=[trace], layout=layout)
fig.show()

# 使用箱线图可视化表中数据分散情况

In [46]:
# go.Box中主要包含六个数据节点，将一组数据从大到小排列，分别计算出他的上边缘，上四分位数Q3，中位数，下四分位数Q1，下边缘，还有一个异常值
y_female = gender_purchase[gender_purchase.Gender == "F"].Purchase
y_male = gender_purchase[gender_purchase.Gender == "M"].Purchase

trace1 = go.Box(y = y_female, name="女性购物", boxmean=True)
trace2 = go.Box(y = y_male, name="男性购物", boxmean=True)
layout = go.Layout(title="消费者的购物分布")
fig = go.Figure(data=[trace1,trace2], layout=layout)
fig.show()

# 分析销售额top10的产品

In [47]:
top10_sellers = data.pivot_table(values="Purchase",
                                index=['Product_ID'],
                                aggfunc="count").reset_index().sort_values(by="Purchase", ascending=False).head(10)
top10_sellers

,Product_ID,Purchase
2534,P00265242,1858
1014,P00110742,1591
249,P00025442,1586
1028,P00112142,1539
565,P00057642,1430
1743,P00184942,1424
458,P00046742,1417
568,P00058042,1396
1353,P00145042,1384
581,P00059442,1384


# 查看购买了top10商品的消费者

In [48]:
top10_seller_buyers = data[data.Product_ID.isin(top10_sellers.Product_ID.to_list())]
top10_seller_buyers

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
13,1000005,P00145042,M,26-35,20,A,1,1,1,2.0,5.0,15665
128,1000025,P00046742,M,18-25,4,C,4+,0,1,2.0,15.0,15770
224,1000042,P00046742,M,26-35,8,C,0,1,1,2.0,15.0,19433
235,1000044,P00112142,M,46-50,17,B,3,1,1,2.0,14.0,19072
...,...,...,...,...,...,...,...,...,...,...,...,...
537405,1004720,P00265242,M,26-35,2,C,4+,0,5,8.0,NaN,8628
537416,1004725,P00265242,M,36-45,5,A,2,0,5,8.0,NaN,6916
537464,1004726,P00058042,F,36-45,16,C,2,1,8,16.0,NaN,7927
537478,1004728,P00265242,M,26-35,19,B,3,0,5,8.0,NaN,6887


# top10产品男人和女人分别都买了多少

In [49]:
top10_seller_gender = top10_seller_buyers.pivot_table(values="Purchase",
                                                     index=["Product_ID","Gender"],
                                                     aggfunc="count").reset_index()
top10_seller_gender

,Product_ID,Gender,Purchase
0,P00025442,F,341
1,P00025442,M,1245
2,P00046742,F,301
3,P00046742,M,1116
4,P00057642,F,256
5,P00057642,M,1174
6,P00058042,F,378
7,P00058042,M,1018
8,P00059442,F,350
9,P00059442,M,1034


# 分析热销产品销量与消费者性别的关系

In [50]:
traces = []
i = 0
for g in top10_seller_gender.Gender.unique():
    trace = go.Bar(x=top10_seller_gender[top10_seller_gender.Gender == g].Purchase,
                    y=top10_seller_gender[top10_seller_gender.Gender == g].Product_ID,
                    name=g,
                    orientation='h')
    traces.append(trace)
    i += 1
layout = go.Layout(title="热销产品销量与消费者性别的关系")
fig = go.Figure(data=traces, layout=layout)
fig.show()
